In [1]:
#only manual input given to file
#note that 1st letter is always upper case and rest of all are lowercase letters
src_name = "Hyderabad"
dst_name = "Tirupati"
#delay_days=1
next_day = "26-Jun-2024"


import requests
import pandas as pd
import datetime
import time
import shlex
import os
import json
import numpy as np
import csv

# List of cities with their IDs
cities = {
    "Mumbai": 462,
    "Hyderabad": 124,
    "Bangalore": 122,
    "Chennai": 123,
    "Vizag": 248,
    "Vijayawada": 134,
    "Guntur": 137,
    "Tirupati": 71756,
    "Madurai": 126,
    "Coimbatore": 141,
    "Trichy": 71929,
    "Pondicherry": 233,
    "Pune": 130,
    "Indore": 313,
    "Nagpur": 624,
    "Goa": 210,
    "Warangal": 95479,
    "Nellore": 131,
    "Kochi": 216,
    "Dharwad": 167,
    "Mysore": 129,
    "Mangaluru": 95222,
    "Thiruvananthapuram": 71425
}
#date_of_extraction = (datetime.datetime.now() + datetime.timedelta(days=delay_days))
#next_day = (datetime.datetime.now() + datetime.timedelta(days=delay_days)).strftime('%d-%b-%Y')
main_filepath=fr"C:\Users\T VIJAYA BALAJI\Desktop\final_data_collection\{src_name}_to_{dst_name}\data_on_{next_day}"
os.makedirs(main_filepath,exist_ok=True)
next_day

'26-Jun-2024'

In [2]:
from datetime import datetime as dt
dummyfilename=f"buses_on_{next_day}"
buses_dir=os.path.join(main_filepath,dummyfilename)
folders= os.listdir(buses_dir)
summary_dir = os.path.join(main_filepath,f'summary_files_on_{next_day}')
os.makedirs(summary_dir, exist_ok=True)


def get_hour_from_foldername(foldername):
    hour_part = foldername.split('.')[0]
    hour_part = dt.strptime(hour_part, "%d-%m-%Y,%H-%M")    
    return dt.strptime(hour_part.strftime("%d-%m-%Y,%H-%M"),"%d-%m-%Y,%H-%M")


for f in folders:
    files = os.listdir(os.path.join(buses_dir,f))
    files.sort(key=get_hour_from_foldername)
     
     
    for i in range(len(files) - 1):
            file1 = files[i]
            file2 = files[i + 1]
            file_path1 = os.path.join(buses_dir, f, file1)
            file_path2 = os.path.join(buses_dir, f, file2)
            hour1=file1.split('.')[0]
            hour2=file2.split('.')[0]
            

            headers = {
            'SEATS': [],
            'STATUS': [],
            'BOOKING TIME': [],
            'REVENUE COLLECTED(only base fare)': []
                    }
            total_df=pd.DataFrame(headers)
            travels_name = f.split('_')[0]
            operator_id=f.split('_')[1]
            route_id=f.split('_')[2]
            summary_filename = f"{travels_name}_{src_name}_to_{dst_name}_data_on_{next_day}_{operator_id}_{route_id}.csv"
            summuary_filepath = os.path.join(summary_dir,summary_filename)
            file_exists = os.path.isfile(summuary_filepath)
            if not file_exists:
                total_df.to_csv(summuary_filepath,index=False)



            # Load the CSV files
            try :
                df1 = pd.read_csv(file_path1)
            except Exception as e:
                print(f"Error reading {file_path1}: {e}")
                continue
            try :
                df2 = pd.read_csv(file_path2)
            except Exception as e:
                print(f"Error reading {file_path2}: {e}")
                continue    
            #df2 = pd.read_csv(file_path2)
            df1 = df1[:-4]
            df2 = df2[:-4]
            
            # Sort data by 'Id'
            df1['id']=df1['seat']
            df2['id']=df2['seat']
            df1.set_index('id',inplace=True)
            df2.set_index('id',inplace=True)
            #print(df1)
            #print(df2)

            df1['occupied'] = pd.to_numeric(df1['occupied'], errors='coerce')
            df1['revenue'] = pd.to_numeric(df1['revenue'], errors='coerce')
            df2['occupied'] = pd.to_numeric(df2['occupied'], errors='coerce')
            df2['revenue'] = pd.to_numeric(df2['revenue'], errors='coerce')

            merged_df = pd.merge(df1, df2, on='seat', suffixes=(f'_on_{hour1}', f'_on_{hour2}'))
            merged_df['id'] = merged_df['seat']
            merged_df.set_index('id',inplace=True)
            merged_df[f'occupied_between_{hour1}_and_{hour2}'] = (df2['occupied'] - df1['occupied']).fillna (0).astype(int)
            #this line is taken if we have dataset instaed of column for  merged_df[f'Fares_Amount_on_{hour1}'] 
            merged_df[f'Fares_Amount_on_{hour1}'] = merged_df[f'Fares_Amount_on_{hour1}'].apply(pd.to_numeric, errors='coerce')
            merged_df[f'occupied_between_{hour1}_and_{hour2}'] = pd.to_numeric(merged_df[f'occupied_between_{hour1}_and_{hour2}'], errors='coerce') 
            
            merged_df[f'revenue_on_seat_between_{hour1}_and_{hour2}'] = merged_df.apply(
                    lambda row: (row[f'Fares_Amount_on_{hour1}'] if row[f'occupied_between_{hour1}_and_{hour2}'] == 1
                    else (-1*row[f'Fares_Amount_on_{hour1}'] if row[f'occupied_between_{hour1}_and_{hour2}'] == -1 else 0)), axis=1)
    #        merged_df= merged_df.drop([f'Fares_Amount_on_{hour1}',f'Fares_Amount_on_{hour2}',f'revenue_on_{hour1}',f'revenue_on_{hour2}',f'Is_Available_on_{hour1}',f'Is_Available_on_{hour2}',f'occupied_on_{hour1}',f'occupied_on_{hour2}'],axis=1)
    #        last_rows=pd.DataFrame({
    #        'Id':[f'Total seat filled between {hour1} and {hour2}',f'Total revenue between {hour1} and {hour2}',f'Cancelled seats between {hour1} and {hour2}'],
    #        f'occupied_between_{hour1}_and_{hour2}':[merged_df[f'occupied_between_{hour1}_and_{hour2}'].astype(float).sum(),merged_df[f'revenue_on_seat_between_{hour1}_and_{hour2}'].astype(float).sum(),merged_df[f'occupied_between_{hour1}_and_{hour2}'].astype(float).eq(-1).sum()],
    #        f'revenue_on_seat_between_{hour1}_and_{hour2}':["","",""]
    
            for index, row in merged_df.iterrows():
                if row[f'occupied_between_{hour1}_and_{hour2}'] == 1:
                    total_df_row = pd.DataFrame({
                        'SEATS': [row['seat']],
                        'STATUS': ["CONFIRMED"],
                        'BOOKING TIME':[f"{hour1} to {hour2}"],
                        'REVENUE COLLECTED(only base fare)':[row[ f'revenue_on_seat_between_{hour1}_and_{hour2}']]
                        })
                    total_df_row.to_csv(summuary_filepath, mode='a', index=False, header=False)
    #            elif row[f'occupied_between_{hour1}_and_{hour2}'] == -1:
    #                total_df_row = pd.DataFrame({
    #                    'SEATS': [row['seat']],
    #                    'STATUS': ["CANCELLED"],
    #                    'BOOKING TIME':[f"{hour1} to {hour2}"],
    #                    'REVENUE COLLECTED(only base fare)':[row[ f'revenue_on_seat_between_{hour1}_and_{hour2}']]
    #                    })
    #                total_df_row.to_csv(summuary_filepath, mode='a', index=False, header=False)
                else: 
                    pass         

    #        merged_df = pd.concat([merged_df, last_rows], ignore_index=True)
            
print(f"sucessesfully completed saving comparisions summary of all buses in route {src_name} to {dst_name} on {next_day}")        

          
        

sucessesfully completed saving comparisions summary of all buses in route Hyderabad to Tirupati on 26-Jun-2024


In [3]:
#3 creating final rows in summary files
summary_dir = os.path.join(main_filepath,f'summary_files_on_{next_day}')
summary_files = [f for f in os.listdir(summary_dir) if f.endswith('.csv')]

for file in summary_files:
        file_path= os.path.join(summary_dir,file)
        total_df=pd.read_csv(file_path)
        total_df = total_df.drop_duplicates(subset=['SEATS'], keep='last')
        total_df.to_csv(file_path, index=False) 

#        total_summary_rows=pd.DataFrame({
#            'SEATS': ["TOTAL SEATS OCCUPIED","TOTAL SEATS CANCELLED","TOTAL REVENUE COLLECTED"],
#            'STATUS': [total_df['STATUS'].value_counts().get('CONFIRMED', 0),total_df['STATUS'].value_counts().get('CANCELLED', 0),total_df['REVENUE COLLECTED(only base fare)'].astype(float).sum()],
#            'BOOKING TIME':[f"","",""],
#            'REVENUE COLLECTED(only base fare)':["","",""]
#        
#        })

        total_summary_rows=pd.DataFrame({
            'SEATS': ["TOTAL SEATS OCCUPIED","TOTAL REVENUE COLLECTED"],
            'STATUS': [total_df['STATUS'].value_counts().get('CONFIRMED', 0),total_df['REVENUE COLLECTED(only base fare)'].astype(float).sum()],
            'BOOKING TIME':[f"",""],
            'REVENUE COLLECTED(only base fare)':["",""]
        
        })

        total_summary_rows.to_csv(file_path, mode='a', index=False, header=False)
        print(total_df)


  SEATS     STATUS                          BOOKING TIME  \
0    L7  CONFIRMED  25-06-2024,01-18 to 25-06-2024,10-18   

   REVENUE COLLECTED(only base fare)  
0                             1300.0  
Empty DataFrame
Columns: [SEATS, STATUS, BOOKING TIME, REVENUE COLLECTED(only base fare)]
Index: []
Empty DataFrame
Columns: [SEATS, STATUS, BOOKING TIME, REVENUE COLLECTED(only base fare)]
Index: []
  SEATS     STATUS                          BOOKING TIME  \
0  06UW  CONFIRMED  24-06-2024,20-18 to 24-06-2024,21-18   
1   05L  CONFIRMED  24-06-2024,22-19 to 24-06-2024,23-18   
2  04LW  CONFIRMED  24-06-2024,22-19 to 24-06-2024,23-18   
3   12L  CONFIRMED  25-06-2024,01-18 to 25-06-2024,10-18   

   REVENUE COLLECTED(only base fare)  
0                             1720.0  
1                             1620.0  
2                             1620.0  
3                             1720.0  
Empty DataFrame
Columns: [SEATS, STATUS, BOOKING TIME, REVENUE COLLECTED(only base fare)]
Index: []
  SEA